In [4]:
# Temporary for correct and eacy insertion of tables
hp_res = {}
bt_res = {}

# Imports

In [5]:
import sys
sys.path.append('/home/si-hj/Desktop/medsegnet')
sys.path.append('/home/si-hj/Desktop')
import numpy as np
from ast import mod
import pathlib
import numpy as np
from omegaconf import DictConfig, OmegaConf
from typing import Dict, Tuple
import torchio as tio
import torch
import matplotlib.pyplot as plt
import os
import data
from data.datasets import MedicalDecathlonDataset, ModalitiesDataset

from pathlib import Path
from time import time
import nibabel as nib
from medsegnet.utils.metrics import dice_coefficient, dice_coefficient_classes, accuracy_score
from medsegnet.utils.utils import setup_seed
from hydra.utils import instantiate
import os
from data.datasets import MedicalDecathlonDataset
import torch.nn.functional as F
from tqdm import tqdm

from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# <span style="color:#fcba03">Data simulation</span>:

### Generel code Blocks

In [6]:
def model_params(model_dir):
	if not model_dir.is_dir():
		raise FileNotFoundError(f"Model directory not found: {model_dir}")
	
	model_path = f"{model_dir}/best_model.pth"
	cfg = OmegaConf.load(f"{model_dir}/config.yaml")
	
	setup_seed(cfg.seed)
	
	if not isinstance(cfg, DictConfig):
		raise TypeError("cfg must be a DictConfig.")

	return model_path, cfg

def pp_list(lst):
	return [f"{dice:.3f}" for dice in lst]

def print_results(dsc_results_per_scale, datasets = None, lowest_dice_plt = False, latex = False):
	dice_scales = []
	class_scales = []
	latex_row = ""
	for scale, results_list in dsc_results_per_scale.items():
		results = np.mean(results_list, axis=1)
		mean_pr_class = np.mean(results_list, axis=0)
		class_scales.append(mean_pr_class)
		std_pr_class = np.std(results_list, axis=0)

		mean_pr_class = [f"{dice:.3f}" for dice in mean_pr_class]
		std_pr_class = [f"{std:.3f}" for std in std_pr_class]

		lowest_idx = None
		lowest_dice = float('inf')

		if datasets is not None and lowest_dice_plt:
			for i, dice in enumerate(results):
				
				if 0.87 < dice and dice < lowest_dice:
					lowest_dice = dice
					lowest_idx = i

		if datasets is not None:
			max_idx, max_dice = datasets[scale] @ np.argmax(results), np.max(results)
			min_idx, min_dice = datasets[scale] @ np.argmin(results), np.min(results)

		
		mean_dice = np.mean(results)
		dice_scales.append(mean_dice)
		std_dice = np.std(results)
		latex_row += f"{mean_dice:.3f} \pm {std_dice:.3f} & "
		str = f"Scale {scale}) Mean Dice: {mean_dice:.3f}\t Std: {std_dice:.3f}\t Mean Pr. Class Dice: {mean_pr_class}\t Std Pr. Class: {std_pr_class}"
		if datasets:
			if lowest_idx is not None and lowest_dice_plt:
				print(f"\tChosen image: {datasets[scale] @ lowest_idx} = {lowest_dice:.3f} DSC")
			print(str + f"\t\tMax Dice({max_idx}): {max_dice:.3f}\t Min Dice({min_idx}): {min_dice:.3f}" if datasets is not None else str)
		else:
			print(str)
	print(f"Mean Dice across all scales: {np.mean(dice_scales):.3f} \t Std: {np.std(dice_scales):.3f}")
	print(f"Mean Dice across all scales of classess: {pp_list(np.mean(class_scales, axis=0))} \t Std: {pp_list(np.std(class_scales, axis=0))}")
	if latex:
		latex_row += f"{np.mean(dice_scales):.3f} \pm {np.std(dice_scales):.3f} \\\\"
		print(f"Latex: {latex_row}")
	
	print()

def consistent_dice(pred, label, num_classes):
	return dice_coefficient_classes(
			pred, label, num_classes, ignore_index=0
		)

def plot_img(image, pred, label):
	image = image[0, 1, :, :, :]
	pred = pred.cpu()

	max_shape = max(image.shape)
	cp = CropOrPad((max_shape, max_shape, max_shape))
	image = cp(image.unsqueeze(0)).squeeze(0)
	label = cp(label[:, :, :].unsqueeze(0)).squeeze(0)
	pred = cp(pred[:, :, :].unsqueeze(0)).squeeze(0)
	

	# Extract slices
	axial_image = np.rot90(image[:, :, image.shape[2] // 2])
	coronal_image = np.rot90(image[:, image.shape[1] // 2, :])
	sagittal_image = np.rot90(image[image.shape[0] // 2, :, :])
	
	axial_label = np.rot90(label[:, :, label.shape[2] // 2])
	coronal_label = np.rot90(label[:, label.shape[1] // 2, :])
	sagittal_label = np.rot90(label[label.shape[0] // 2, :, :])

	
	axial_pred = np.rot90(pred[:, :, pred.shape[2] // 2])
	coronal_pred = np.rot90(pred[:, pred.shape[1] // 2, :])
	sagittal_pred = np.rot90(pred[pred.shape[0] // 2, :, :])

	images = [
		axial_image, axial_label, axial_pred,
		coronal_image, coronal_label, coronal_pred,
		sagittal_image, sagittal_label, sagittal_pred
	]

	titles_x = ["Image", "Label", "Prediction"]
	titles_y = ["Axial", "Coronal", "Sagittal"]

	
	fig, axs = plt.subplots(3, 3, figsize=(9, 9))
	for i in range(3):
		for j in range(3):
			axs[i, j].imshow(images[i * 3 + j], cmap="gray")
			# axs[i, j].axis("off")
			if i == 0:
				axs[i, j].set_title(titles_x[j])
				
			if j == 0:
				label_obj = axs[i, j].set_ylabel(titles_y[i], rotation=90, labelpad=20)
				label_obj.set_verticalalignment('center')
	
	plt.tight_layout()
	plt.show()

In [ ]:
_dataset_config = {
    "Task01_BrainTumour": {
        "best_models_paths": {
            "bb": "trained_models/unet3d/Task01_BrainTumour/2025-07-01_02:56:40_Brats-Baseline-UNet",
            "aug": "trained_models/unet-aug3d/Task01_BrainTumour/2025-07-01_02:56:40_Brats-Baseline-UNet-Aug", 
            "ms": "trained_models/ms-unet3d/Task01_BrainTumour/2025-07-01_02:56:40_Brats-RAREUNet-Final"    
        },
    },
    "Task04_Hippocampus": {
        "best_models_paths": {
            "bb": "trained_models/unet3d/Task04_Hippocampus/2025-07-01_02:56:37_Hippo-Baseline-UNet",
            "aug": "trained_models/unet-aug3d/Task04_Hippocampus/2025-07-01_02:56:37_Hippo-Baseline-UNet-Aug",
            "ms": "trained_models/ms-unet3d/Task04_Hippocampus/2025-07-01_02:56:37_Hippo-RAREUNet-Final"
        },
    },
    "Task05_Prostate": {
        "best_models_paths": {
            "bb": "",
            "aug": "",
            "ms": "" # "trained_models/ms-unet3d/Task05_Prostate/2025-06-14_18:28:37_test_prostate_instance/"
        },
    }
}

### DataSimulation -- Our model

In [8]:
import warnings

from networkx import center
import test
from torchio import CropOrPad
warnings.filterwarnings("ignore", category=UserWarning)

def test_lowres(model_dir_str: str, desc: str = "", classes: bool = True, plot: bool = False, num_ms_levels: int = 3):
	if model_dir_str is None or model_dir_str == "":
		print("No model directory provided.")
		return
	if not model_dir_str.startswith("/home/si-hj/Desktop/medsegnet/"):
		model_dir_str = "/home/si-hj/Desktop/medsegnet/" + model_dir_str
	model_dir = Path(model_dir_str)
	model_path, cfg = model_params(model_dir)

	scales = list(range(num_ms_levels+1))


	base_dir = Path(cfg.dataset.base_path)
	if not str(base_dir).endswith("_test1"):
		base_dir = Path(f"{base_dir}_test1")
		
	lowres_dir = base_dir / "lowres" / "downsampled"

	datasets = {}

	for scale in scales:
		img_dir = lowres_dir / f"scale{scale}" / "imagesTs"
		lbl_dir = lowres_dir / f"scale{scale}" / "labelsTs"
		img_dir_sort = sorted(os.listdir(img_dir))
		lbl_dir_sort = sorted(os.listdir(lbl_dir))
		if (cfg.dataset.name == "Task04_Hippocampus"):
			dataset = MedicalDecathlonDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
		else:
			dataset = ModalitiesDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
		datasets[scale] = dataset


	
	model = instantiate(cfg.architecture.path, cfg, "inference")

	checkpoint = torch.load(model_path, map_location=device)
	model_state_dict = checkpoint["model_state_dict"]
	model.load_state_dict(model_state_dict)
	model.to(device)
	model.eval()

	dsc_results_per_scale = {scale: [] for scale in scales}

	print(f"******************* TEST LowRes {desc + ' ' if desc != '' else ''}*******************")
	for scale in scales:
		with torch.no_grad():
			dataloader = DataLoader(datasets[scale], batch_size=1, shuffle=False, num_workers=cfg.training.num_workers)
			for idx, (image, label) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Scale {scale}"):
				
				output = model.run_inference(image.to(device))
				pred = torch.argmax(output, dim=1).squeeze(0)
				label = label.squeeze(0)
				
				dsc_pr_class = consistent_dice(pred, label.to(device), cfg.dataset.num_classes)
				dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])

				if plot and (scale == 0 and np.mean(dsc_pr_class) <= 0.3 or np.mean(dsc_pr_class) < 0.1):
					print(f"dice: {dsc_pr_class.item()}\t{datasets[scale] @ idx}")
					plot_img(image, pred, label)
						

	print(f"Results per scale of lowres:")
	print_results(dsc_results_per_scale, datasets, lowest_dice_plt=False, latex=True)
	return dsc_results_per_scale

for path, task in [
	# ("trained_models/ms-unet3d/Task01_BrainTumour/2025-06-13_18:28:57_msinstance_0dropout_msgate-instancenorm", "bt - check"),
 	# ("trained_models/ms-unet3d/Task01_BrainTumour/2025-06-13_03:34:22_msinstance_full", "bt - test")	
]:
	if not path.startswith("/home/si-hj/Desktop/medsegnet/") and path != "":
		path = "/home/si-hj/Desktop/medsegnet/" + path
	if not os.path.exists(path):
		print(f"Path does not exist: {path}")
		continue
	test_lowres(path, f"quciktest | Test runs {task} Models (Multiscale)")

# Hippocampus
print(f"{'*' * 20} HippoCampus {'*' * 20}")
hp_res["our"] = test_lowres(_dataset_config['Task04_Hippocampus']['best_models_paths'].get('ms'), "HippoCampus | Best Model (Multiscale)")

# Prostate
print(f"{'*' * 20} Prostate {'*' * 20}")
# test_lowres(_dataset_config['Task05_Prostate']['best_models_paths'].get('ms'), "Prostate | Best Model (Multiscale)")

# BrainTumour
print(f"{'*' * 20} BrainTumour {'*' * 20}")
#bt_res["our"] = test_lowres(_dataset_config['Task01_BrainTumour']['best_models_paths'].get('ms'), "BrainTumour | Best Model (Multiscale)")



INFO: training = True
INFO: depth = 4
INFO: target_shape = [32, 64, 32]
INFO: in_channels = 1
INFO: n_filters = 15
INFO: dropout = 0.1
INFO: num_classes = 3
INFO: norm_type = instance
INFO: activation_type = leaky_relu


******************** HippoCampus ********************
MSUNet3D: MODE=inference
******************* TEST LowRes HippoCampus | Best Model (Multiscale) *******************


Scale 3: 100%|██████████| 52/52 [00:00<00:00, 100.59it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.870	 Std: 0.034	 Mean Pr. Class Dice: ['0.878', '0.862']	 Std Pr. Class: ['0.035', '0.040']		Max Dice(hippocampus_227.nii.gz): 0.928	 Min Dice(hippocampus_282.nii.gz): 0.775
Scale 1) Mean Dice: 0.861	 Std: 0.040	 Mean Pr. Class Dice: ['0.869', '0.853']	 Std Pr. Class: ['0.042', '0.042']		Max Dice(hippocampus_065.nii.gz): 0.923	 Min Dice(hippocampus_320.nii.gz): 0.735
Scale 2) Mean Dice: 0.823	 Std: 0.062	 Mean Pr. Class Dice: ['0.839', '0.808']	 Std Pr. Class: ['0.075', '0.067']		Max Dice(hippocampus_044.nii.gz): 0.928	 Min Dice(hippocampus_199.nii.gz): 0.652
Scale 3) Mean Dice: 0.771	 Std: 0.101	 Mean Pr. Class Dice: ['0.797', '0.744']	 Std Pr. Class: ['0.142', '0.166']		Max Dice(hippocampus_044.nii.gz): 1.000	 Min Dice(hippocampus_199.nii.gz): 0.500
Mean Dice across all scales: 0.831 	 Std: 0.039
Mean Dice across all scales of classess: ['0.846', '0.817'] 	 Std: ['0.032', '0.047']
Latex: 0.870 \pm 0.034 & 0.861 \pm 0.040 & 0.823 \pm 

### DataSimulation -- BackBone

In [9]:

def test_backbone(model_dir_str, desc: str = "", depth: int = 3):
	if model_dir_str is None or model_dir_str == "":
		print("No model directory provided.")
		return
	if not model_dir_str.startswith("/home/si-hj/Desktop/medsegnet/"):
		model_dir_str = "/home/si-hj/Desktop/medsegnet/" + model_dir_str
	model_dir = Path(model_dir_str)
	model_path, cfg = model_params(model_dir)
	
	scales = list(range(depth+1))


	base_dir = Path(cfg.dataset.base_path)
	if not str(base_dir).endswith("_test1"):
		base_dir = Path(f"{base_dir}_test1")

	pad_dir = base_dir / "fullres" / "pad"
	upsampling_dir = base_dir / "fullres" / "upsampled"
	datasets = {}

	def get_dataset(base_name_dir, name):
		datasets[name] = {}
		for scale in scales:
			img_dir = os.path.join(base_name_dir, f"scale{scale}", "imagesTs")
			lbl_dir = os.path.join(base_dir, "lowres", "downsampled", f"scale{scale}", "labelsTs")
			img_dir_sort = sorted(os.listdir(img_dir))
			lbl_dir_sort = sorted(os.listdir(lbl_dir))
			if (cfg.dataset.name == "Task01_BrainTumour"):
				dataset = ModalitiesDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
			else:
				dataset = MedicalDecathlonDataset(cfg, "test", img_dir_sort, lbl_dir_sort, str(img_dir), str(lbl_dir))
			datasets[name][scale] = dataset

	get_dataset(pad_dir, "pad")
	get_dataset(upsampling_dir, "upsampled")

	model = instantiate(cfg.architecture.path, cfg, mode="inference")

	checkpoint = torch.load(model_path, map_location=device)
	model_state_dict = checkpoint["model_state_dict"]
	model.load_state_dict(model_state_dict)
	model.to(device)
	model.eval()

	# dsc_results_per_scale = {scale: [] for scale in scales} moved this inside so its not averaged over pad and upsampled....
	print(f"******************* TEST Pad/upsamp {desc + ' ' if desc != '' else ''}*******************")
	def test(dataset, name):
		dsc_results_per_scale = {scale: [] for scale in scales} 
		for scale in scales:
			with torch.no_grad():
				for idx, (image, label) in tqdm(enumerate(dataset[name][scale]), desc=f"Scale {scale}", total=len(dataset[name][scale])):
					
					x = image.unsqueeze(0).float().to(device)

					output = model.forward(x)

					output = output.argmax(dim=1).unsqueeze(0) 

					target_shape = tuple(label.shape)

					if name == "pad": # crop to target shape
						cp = tio.CropOrPad(target_shape, padding_mode="constant")
						tio_image = tio.ScalarImage(tensor=output.squeeze(0))

						scaled_output = cp(tio_image)
						scaled_output = (scaled_output.data).unsqueeze(0)
					else: # downsample to target shape
						scaled_output = F.interpolate(
							output.float(), size=target_shape, mode="nearest"
						)

					label = label.unsqueeze(0)

					pred = scaled_output.squeeze(0)
					# pred = torch.argmax(scaled_output, dim=1)

					dsc_pr_class = consistent_dice(pred.to(device), label.to(device), cfg.dataset.num_classes)
					dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])

					
					
		print(f"Results per scale of {name}:")
		print_results(dsc_results_per_scale, dataset[name], latex=True)
		return dsc_results_per_scale
		
	pad_res = test(datasets, "pad")
	up_res = test(datasets, "upsampled")
	print("\n")
	return pad_res, up_res


# Hippocampus
print(f"{'*' * 20} HippoCampus {'*' * 20}")
hp_res["bb_pad"], hp_res["bb_up"] = test_backbone(_dataset_config['Task04_Hippocampus']['best_models_paths'].get('bb'), "HippoCampus | Best U-Net3D")
hp_res["bb-aug_pad"], hp_res["bb-aug_up"] = test_backbone(_dataset_config['Task04_Hippocampus']['best_models_paths'].get('aug'), "HippoCampus | Best U-Net-aug3D")

# Prostate
print(f"{'*' * 20} Prostate {'*' * 20}")
# test_backbone(_dataset_config['Task05_Prostate']['best_models_paths'].get('bb'), "Prostate | Best U-Net3D")
# test_backbone(_dataset_config['Task05_Prostate']['best_models_paths'].get('aug'), "Prostate | Best U-Net-aug3D")

# Braintumour
print(f"{'*' * 20} BrainTumour {'*' * 20}")
#bt_res["bb_pad"], bt_res["bb_up"] = test_backbone(_dataset_config['Task01_BrainTumour']['best_models_paths'].get('bb'), "BrainTumour | Best U-Net3D")
#bt_res["bb-aug_pad"], bt_res["bb-aug_up"] = test_backbone(_dataset_config['Task01_BrainTumour']['best_models_paths'].get('aug'), "BrainTumour | Best U-Net-aug3D")

INFO: training = True
INFO: depth = 4
INFO: target_shape = [32, 64, 32]
INFO: in_channels = 1
INFO: n_filters = 15
INFO: dropout = 0.1
INFO: num_classes = 3
INFO: norm_type = instance
INFO: activation_type = leaky_relu


******************** HippoCampus ********************
******************* TEST Pad/upsamp HippoCampus | Best U-Net3D *******************


Scale 3:   0%|          | 0/52 [00:00<?, ?it/s]/home/si-hj/.conda/envs/bp/lib/python3.10/site-packages/torchio/transforms/preprocessing/intensity/rescale.py:89: RuntimeWarning: Rescaling image "image" not possible because all the intensity values are the same
  image.set_data(self.rescale(image.data, mask, image_name))
Scale 3: 100%|██████████| 52/52 [00:03<00:00, 15.91it/s]


Results per scale of pad:
Scale 0) Mean Dice: 0.879	 Std: 0.033	 Mean Pr. Class Dice: ['0.887', '0.870']	 Std Pr. Class: ['0.033', '0.039']		Max Dice(hippocampus_373.nii.gz): 0.927	 Min Dice(hippocampus_282.nii.gz): 0.795
Scale 1) Mean Dice: 0.196	 Std: 0.064	 Mean Pr. Class Dice: ['0.106', '0.286']	 Std Pr. Class: ['0.096', '0.055']		Max Dice(hippocampus_223.nii.gz): 0.366	 Min Dice(hippocampus_274.nii.gz): 0.071
Scale 2) Mean Dice: 0.115	 Std: 0.039	 Mean Pr. Class Dice: ['0.139', '0.092']	 Std Pr. Class: ['0.029', '0.071']		Max Dice(hippocampus_227.nii.gz): 0.226	 Min Dice(hippocampus_003.nii.gz): 0.062
Scale 3) Mean Dice: 0.048	 Std: 0.039	 Mean Pr. Class Dice: ['0.077', '0.019']	 Std Pr. Class: ['0.037', '0.066']		Max Dice(hippocampus_321.nii.gz): 0.183	 Min Dice(hippocampus_172.nii.gz): 0.000
Mean Dice across all scales: 0.310 	 Std: 0.333
Mean Dice across all scales of classess: ['0.302', '0.317'] 	 Std: ['0.338', '0.334']
Latex: 0.879 \pm 0.033 & 0.196 \pm 0.064 & 0.115 \pm 0.0

Scale 3: 100%|██████████| 52/52 [00:02<00:00, 21.50it/s]
INFO: training = True
INFO: depth = 4
INFO: target_shape = [32, 64, 32]
INFO: in_channels = 1
INFO: n_filters = 15
INFO: dropout = 0.1
INFO: num_classes = 3
INFO: norm_type = instance
INFO: activation_type = leaky_relu


Results per scale of upsampled:
Scale 0) Mean Dice: 0.879	 Std: 0.033	 Mean Pr. Class Dice: ['0.887', '0.870']	 Std Pr. Class: ['0.033', '0.039']		Max Dice(hippocampus_373.nii.gz): 0.927	 Min Dice(hippocampus_282.nii.gz): 0.795
Scale 1) Mean Dice: 0.843	 Std: 0.050	 Mean Pr. Class Dice: ['0.855', '0.832']	 Std Pr. Class: ['0.049', '0.066']		Max Dice(hippocampus_065.nii.gz): 0.919	 Min Dice(hippocampus_173.nii.gz): 0.683
Scale 2) Mean Dice: 0.741	 Std: 0.090	 Mean Pr. Class Dice: ['0.747', '0.735']	 Std Pr. Class: ['0.087', '0.120']		Max Dice(hippocampus_065.nii.gz): 0.881	 Min Dice(hippocampus_199.nii.gz): 0.406
Scale 3) Mean Dice: 0.293	 Std: 0.141	 Mean Pr. Class Dice: ['0.497', '0.088']	 Std Pr. Class: ['0.232', '0.177']		Max Dice(hippocampus_320.nii.gz): 0.583	 Min Dice(hippocampus_274.nii.gz): 0.000
Mean Dice across all scales: 0.689 	 Std: 0.234
Mean Dice across all scales of classess: ['0.747', '0.631'] 	 Std: ['0.153', '0.317']
Latex: 0.879 \pm 0.033 & 0.843 \pm 0.050 & 0.741 \

Scale 3: 100%|██████████| 52/52 [00:03<00:00, 15.94it/s]


Results per scale of pad:
Scale 0) Mean Dice: 0.876	 Std: 0.033	 Mean Pr. Class Dice: ['0.883', '0.869']	 Std Pr. Class: ['0.033', '0.036']		Max Dice(hippocampus_373.nii.gz): 0.925	 Min Dice(hippocampus_223.nii.gz): 0.807
Scale 1) Mean Dice: 0.230	 Std: 0.042	 Mean Pr. Class Dice: ['0.270', '0.190']	 Std Pr. Class: ['0.054', '0.045']		Max Dice(hippocampus_368.nii.gz): 0.326	 Min Dice(hippocampus_350.nii.gz): 0.146
Scale 2) Mean Dice: 0.042	 Std: 0.032	 Mean Pr. Class Dice: ['0.060', '0.023']	 Std Pr. Class: ['0.049', '0.028']		Max Dice(hippocampus_373.nii.gz): 0.109	 Min Dice(hippocampus_040.nii.gz): 0.000
Scale 3) Mean Dice: 0.000	 Std: 0.003	 Mean Pr. Class Dice: ['0.000', '0.001']	 Std Pr. Class: ['0.000', '0.005']		Max Dice(hippocampus_242.nii.gz): 0.019	 Min Dice(hippocampus_003.nii.gz): 0.000
Mean Dice across all scales: 0.287 	 Std: 0.351
Mean Dice across all scales of classess: ['0.303', '0.271'] 	 Std: ['0.349', '0.353']
Latex: 0.876 \pm 0.033 & 0.230 \pm 0.042 & 0.042 \pm 0.0

Scale 3: 100%|██████████| 52/52 [00:02<00:00, 21.40it/s]

Results per scale of upsampled:
Scale 0) Mean Dice: 0.876	 Std: 0.033	 Mean Pr. Class Dice: ['0.883', '0.869']	 Std Pr. Class: ['0.033', '0.036']		Max Dice(hippocampus_373.nii.gz): 0.925	 Min Dice(hippocampus_223.nii.gz): 0.807
Scale 1) Mean Dice: 0.863	 Std: 0.040	 Mean Pr. Class Dice: ['0.871', '0.856']	 Std Pr. Class: ['0.039', '0.046']		Max Dice(hippocampus_044.nii.gz): 0.933	 Min Dice(hippocampus_173.nii.gz): 0.766
Scale 2) Mean Dice: 0.826	 Std: 0.065	 Mean Pr. Class Dice: ['0.839', '0.812']	 Std Pr. Class: ['0.078', '0.073']		Max Dice(hippocampus_094.nii.gz): 0.950	 Min Dice(hippocampus_199.nii.gz): 0.650
Scale 3) Mean Dice: 0.791	 Std: 0.130	 Mean Pr. Class Dice: ['0.783', '0.798']	 Std Pr. Class: ['0.148', '0.196']		Max Dice(hippocampus_094.nii.gz): 1.000	 Min Dice(hippocampus_173.nii.gz): 0.365
Mean Dice across all scales: 0.839 	 Std: 0.033
Mean Dice across all scales of classess: ['0.844', '0.834'] 	 Std: ['0.039', '0.029']
Latex: 0.876 \pm 0.033 & 0.863 \pm 0.040 & 0.826 \

### DataSimulation -- NNUNet

In [10]:
from pprint import pp
from sympy import factor
import yaml


def test_nnUNet(pred_dir_str, desc: str = "", depth: int = 3):
	scales = list(range(depth+1))

	dataset_short_name, dataset_type = pred_dir_str.split("/")

	dsc_results_per_scale = {scale: [] for scale in scales}
	print(f"TEST LowRes{' ' + desc if desc != '' else ''}:")
	dataset = {
		"hp": "Task04_Hippocampus",
		"bt": "Task01_BrainTumour",
		"p": "Task05_Prostate",
	}
	dataset_name = dataset[dataset_short_name]

	for scale in scales:
		label_name_str = f"/home/si-hj/Desktop/datasets/{dataset_name}_test1/lowres/downsampled/scale{scale}/labelsTs" 
		label_name_dir = Path(label_name_str)

		pred_name_str = f"/home/si-hj/Desktop/nn-unet/predictions/{pred_dir_str}/scale{scale}"
		pred_name_dir = Path(pred_name_str)
		
		image_files = sorted([f for f in os.listdir(pred_name_dir) if f.endswith(".nii.gz")])
		label_files = sorted([f for f in os.listdir(label_name_dir) if f.endswith(".nii.gz")])




		for img_file, label_file in tqdm(zip(image_files, label_files), desc=f"Scale {scale}", total=len(image_files)):
			img_path = pred_name_dir / img_file
			label_path = label_name_dir / label_file
			

			image = nib.load(str(img_path)).get_fdata() # (W, H, D)
			scaled_label = nib.load(str(label_path)).get_fdata() # (W, H, D)

			factor = 1 / (2 ** scale)
			if dataset_type == "pad":
				# target shape
				target_shape = tuple(round(dim * factor) for dim in image.shape)

				tio_image = tio.ScalarImage(str(img_path)) # (C, W, H, D)
				cp = tio.CropOrPad(target_shape)
				tio_image = cp(tio_image)
				scaled_image = torch.from_numpy(tio_image.data.numpy()).squeeze(0) # (C, W, H, D) -> (C, D, H, W)

			elif dataset_type == "upsampled":
				tensor = torch.from_numpy(image).unsqueeze(0).unsqueeze(0) # (W, H, D) -> (1, 1, D, H, W)
				tensor_out = F.interpolate(tensor, scale_factor=factor, mode="nearest")
			
				scaled_image = tensor_out.squeeze(0).squeeze(0)
			else: 
				raise ValueError(f"Unknown dataset type: {dataset_type} (available: pad, upsampled)")
			
			scaled_label = torch.from_numpy(scaled_label)

			# get medsegnet/config/dataset/Task01_BrainTumour.yaml -> num_classes
			num_classes = yaml.safe_load(open(f"/home/si-hj/Desktop/medsegnet/config/dataset/{dataset_name}.yaml"))['num_classes']
			
			dsc_pr_class = consistent_dice(
				scaled_image.to(device), scaled_label.to(device), num_classes
			)
			dsc_results_per_scale[scale].append([dsc_class.item() for dsc_class in dsc_pr_class])
			
	print(f"Results per scale of lowres:")
	print_results(dsc_results_per_scale, latex=True)
	return dsc_results_per_scale
			


# Hippocampus
print(f"{'*' * 20} HippocCampus {'*' * 20}")
hp_res["nn_pad"] = test_nnUNet(f"hp/pad", f"HP - Pad Best achieved for nnU-net")
hp_res["nn_up"] = test_nnUNet(f"hp/upsampled", f"HP - Upsampled Best achieved for nnU-net")

# Prostate
# print(f"{'*' * 20} Prostate {'*' * 20}")
# test_nnUNet(f"p/pad", f"P - Pad | Best achieved for nnU-net")
# test_nnUNet(f"p/upsampled", f"P - Upsampled | Best achieved for nnU-net")


# Braintumour
print(f"{'*' * 20} BrainTumour {'*' * 20}")
bt_res["nn_pad"] = test_nnUNet(f"bt/pad", f"BT - Pad | Best achieved for nnU-net")
bt_res["nn_up"] = test_nnUNet(f"bt/upsampled", f"BT - Upsampled | Best achieved for nnU-net")

******************** HippocCampus ********************
TEST LowRes HP - Pad Best achieved for nnU-net:


Scale 3: 100%|██████████| 52/52 [00:01<00:00, 35.93it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.880	 Std: 0.032	 Mean Pr. Class Dice: ['0.889', '0.871']	 Std Pr. Class: ['0.033', '0.034']
Scale 1) Mean Dice: 0.819	 Std: 0.045	 Mean Pr. Class Dice: ['0.835', '0.804']	 Std Pr. Class: ['0.038', '0.063']
Scale 2) Mean Dice: 0.529	 Std: 0.097	 Mean Pr. Class Dice: ['0.552', '0.505']	 Std Pr. Class: ['0.111', '0.113']
Scale 3) Mean Dice: 0.030	 Std: 0.062	 Mean Pr. Class Dice: ['0.054', '0.005']	 Std Pr. Class: ['0.120', '0.039']
Mean Dice across all scales: 0.564 	 Std: 0.336
Mean Dice across all scales of classess: ['0.582', '0.546'] 	 Std: ['0.331', '0.341']
Latex: 0.880 \pm 0.032 & 0.819 \pm 0.045 & 0.529 \pm 0.097 & 0.030 \pm 0.062 & 0.564 \pm 0.336 \\

TEST LowRes HP - Upsampled Best achieved for nnU-net:


Scale 3: 100%|██████████| 52/52 [00:00<00:00, 340.01it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.880	 Std: 0.032	 Mean Pr. Class Dice: ['0.889', '0.871']	 Std Pr. Class: ['0.033', '0.034']
Scale 1) Mean Dice: 0.820	 Std: 0.067	 Mean Pr. Class Dice: ['0.831', '0.808']	 Std Pr. Class: ['0.066', '0.075']
Scale 2) Mean Dice: 0.597	 Std: 0.106	 Mean Pr. Class Dice: ['0.586', '0.609']	 Std Pr. Class: ['0.135', '0.121']
Scale 3) Mean Dice: 0.055	 Std: 0.085	 Mean Pr. Class Dice: ['0.087', '0.023']	 Std Pr. Class: ['0.135', '0.081']
Mean Dice across all scales: 0.588 	 Std: 0.325
Mean Dice across all scales of classess: ['0.598', '0.578'] 	 Std: ['0.316', '0.334']
Latex: 0.880 \pm 0.032 & 0.820 \pm 0.067 & 0.597 \pm 0.106 & 0.055 \pm 0.085 & 0.588 \pm 0.325 \\

******************** BrainTumour ********************
TEST LowRes BT - Pad | Best achieved for nnU-net:


Scale 3: 100%|██████████| 97/97 [00:08<00:00, 11.03it/s]


Results per scale of lowres:
Scale 0) Mean Dice: 0.712	 Std: 0.152	 Mean Pr. Class Dice: ['0.794', '0.591', '0.752']	 Std Pr. Class: ['0.112', '0.253', '0.243']
Scale 1) Mean Dice: 0.670	 Std: 0.152	 Mean Pr. Class Dice: ['0.765', '0.537', '0.706']	 Std Pr. Class: ['0.120', '0.256', '0.248']
Scale 2) Mean Dice: 0.583	 Std: 0.128	 Mean Pr. Class Dice: ['0.682', '0.468', '0.598']	 Std Pr. Class: ['0.133', '0.234', '0.230']
Scale 3) Mean Dice: 0.399	 Std: 0.125	 Mean Pr. Class Dice: ['0.491', '0.330', '0.377']	 Std Pr. Class: ['0.169', '0.211', '0.237']
Mean Dice across all scales: 0.591 	 Std: 0.120
Mean Dice across all scales of classess: ['0.683', '0.482', '0.608'] 	 Std: ['0.119', '0.098', '0.145']
Latex: 0.712 \pm 0.152 & 0.670 \pm 0.152 & 0.583 \pm 0.128 & 0.399 \pm 0.125 & 0.591 \pm 0.120 \\

TEST LowRes BT - Upsampled | Best achieved for nnU-net:


Scale 3: 100%|██████████| 97/97 [00:04<00:00, 23.59it/s]

Results per scale of lowres:
Scale 0) Mean Dice: 0.712	 Std: 0.152	 Mean Pr. Class Dice: ['0.794', '0.591', '0.752']	 Std Pr. Class: ['0.112', '0.253', '0.243']
Scale 1) Mean Dice: 0.660	 Std: 0.155	 Mean Pr. Class Dice: ['0.750', '0.518', '0.711']	 Std Pr. Class: ['0.127', '0.255', '0.250']
Scale 2) Mean Dice: 0.595	 Std: 0.140	 Mean Pr. Class Dice: ['0.698', '0.444', '0.644']	 Std Pr. Class: ['0.137', '0.244', '0.253']
Scale 3) Mean Dice: 0.452	 Std: 0.145	 Mean Pr. Class Dice: ['0.553', '0.321', '0.482']	 Std Pr. Class: ['0.185', '0.233', '0.288']
Mean Dice across all scales: 0.605 	 Std: 0.097
Mean Dice across all scales of classess: ['0.699', '0.469', '0.647'] 	 Std: ['0.091', '0.100', '0.103']
Latex: 0.712 \pm 0.152 & 0.660 \pm 0.155 & 0.595 \pm 0.140 & 0.452 \pm 0.145 & 0.605 \pm 0.097 \\



# Temporary to print tables

This one is for overall mean

In [11]:
import numpy as np

def generate_latex_table(results_dict, model_names_map, table_title, scales_order=[0, 1, 2, 3]):
    """
    Generates and prints LaTeX table rows from a dictionary of results.
    It highlights the best (bold) and second-best (underline) values in each column
    by comparing values rounded to 3 decimal places, correctly handling all ties.

    Args:
        results_dict (dict): The dictionary containing the model results (e.g., hp_res, bt_res).
        model_names_map (dict): A map from keys in results_dict to display names for the table.
        table_title (str): The title to print above the table.
        scales_order (list): The order of scales to process.
    """
    print(f"\n\n{'='*20} LaTeX Table Rows for {table_title} {'='*20}")
    
    # --- PASS 1: Pre-compute all data ---
    all_models_data = []
    model_keys_in_order = [key for key in model_names_map.keys() if key in results_dict and results_dict[key] is not None]

    for model_key in model_keys_in_order:
        dsc_results_per_scale = results_dict[model_key]
        if not all(s in dsc_results_per_scale for s in scales_order):
            continue
        
        model_means = []
        model_stds = []
        for scale in scales_order:
            results_list = dsc_results_per_scale[scale]
            results_mean_per_image = np.mean(results_list, axis=1)
            model_means.append(np.mean(results_mean_per_image))
            model_stds.append(np.std(results_mean_per_image))

        overall_mean = np.mean(model_means)
        overall_std = np.std(model_means)
        model_means.append(overall_mean)
        model_stds.append(overall_std)
        
        all_models_data.append({
            'key': model_key,
            'name': model_names_map[model_key],
            'means': model_means,
            'stds': model_stds
        })

    if not all_models_data:
        print(f"No valid data found for table: {table_title}")
        return

    # --- PASS 2: Analyze columns using rounded values and generate table ---
    means_array = np.array([data['means'] for data in all_models_data])
    num_cols = means_array.shape[1]
    
    max_vals_per_col = []
    second_max_vals_per_col = []

    for j in range(num_cols):
        # --- ROBUST LOGIC USING ROUNDING ---
        # 1. Round all values in the column to 3 decimal places for comparison
        col_values_rounded = np.round(means_array[:, j], 3)
        
        # 2. Get the unique sorted values from the rounded data
        unique_sorted_desc = np.unique(col_values_rounded)[::-1]
        
        # 3. Assign the top two unique rounded values
        max_val = unique_sorted_desc[0] if len(unique_sorted_desc) > 0 else -1.0
        second_max_val = unique_sorted_desc[1] if len(unique_sorted_desc) > 1 else -1.0
        
        max_vals_per_col.append(max_val)
        second_max_vals_per_col.append(second_max_val)

    # --- Print Data Rows ---
    for i, model_data in enumerate(all_models_data):
        latex_parts = []
        for j in range(num_cols):
            mean = model_data['means'][j]
            std = model_data['stds'][j]
            
            # Use the original mean for display formatting, but the rounded mean for logic
            mean_for_comparison = round(mean, 3)
            mean_str_for_display = f"{mean:.3f}"
            
            max_val = max_vals_per_col[j]
            second_max_val = second_max_vals_per_col[j]

            # Compare the rounded value against the stored rounded max/second_max
            if mean_for_comparison == max_val:
                formatted_mean = f"\\textbf{{{mean_str_for_display}}}"
            elif mean_for_comparison == second_max_val:
                formatted_mean = f"\\underline{{{mean_str_for_display}}}"
            else:
                formatted_mean = mean_str_for_display
            
            latex_parts.append(f"{formatted_mean}$\\pm${std:.3f}")
        
        row_content = " & ".join(latex_parts)
        
        model_key = model_data['key']
        
        if model_key.endswith('_up') and i < len(all_models_data) - 1:
             print(f"{model_data['name']} & {row_content} \\\\\\midrule")
        else:
             print(f"{model_data['name']} & {row_content} \\\\")

    print(f"{'='*20} End of Table {'='*20}\n")


# The rest of your script remains the same
model_names = {
    "bb_pad": "UNet--Pad",
    "bb_up": "UNet--Up",
    "bb-aug_pad": "UNet+Aug--Pad",
    "bb-aug_up": "UNet+Aug--Up",
    "nn_pad": "nnUNet--Pad",
    "nn_up": "nnUNet--Up",
    "our": "RARE-UNet",
}

# The rest of your script, including where you call this function, remains the same.
generate_latex_table(hp_res, model_names, "Hippocampus")
generate_latex_table(bt_res, model_names, "BrainTumour")



==================== LaTeX Table Rows for Hippocampus ====================
UNet--Pad & \underline{0.879}$\pm$0.033 & 0.196$\pm$0.064 & 0.115$\pm$0.039 & 0.048$\pm$0.039 & 0.310$\pm$0.333 \\
UNet--Up & \underline{0.879}$\pm$0.033 & 0.843$\pm$0.050 & 0.741$\pm$0.090 & 0.293$\pm$0.141 & 0.689$\pm$0.234 \\\midrule
UNet+Aug--Pad & 0.876$\pm$0.033 & 0.230$\pm$0.042 & 0.042$\pm$0.032 & 0.000$\pm$0.003 & 0.287$\pm$0.351 \\
UNet+Aug--Up & 0.876$\pm$0.033 & \textbf{0.863}$\pm$0.040 & \textbf{0.826}$\pm$0.065 & \textbf{0.791}$\pm$0.130 & \textbf{0.839}$\pm$0.033 \\\midrule
nnUNet--Pad & \textbf{0.880}$\pm$0.032 & 0.819$\pm$0.045 & 0.529$\pm$0.097 & 0.030$\pm$0.062 & 0.564$\pm$0.336 \\
nnUNet--Up & \textbf{0.880}$\pm$0.032 & 0.820$\pm$0.067 & 0.597$\pm$0.106 & 0.055$\pm$0.085 & 0.588$\pm$0.325 \\\midrule
RARE-UNet & 0.870$\pm$0.034 & \underline{0.861}$\pm$0.040 & \underline{0.823}$\pm$0.062 & \underline{0.771}$\pm$0.101 & \underline{0.831}$\pm$0.039 \\
==================== End of Table =========

This one is for all the classes

In [12]:
import numpy as np

def generate_class_table_grouped(hp_res, bt_res):
    """
    Generates a LaTeX table for per-class Dice scores, applying formatting
    (bold for best, underline for second-best) INDEPENDENTLY within each scale group.
    Labels are formatted as 'Model--Pad' and 'Model--Up'.
    """
    
    # --- Configuration: Define the structure of the table ---
    columns = [
        ('HP--L', hp_res, 0), ('HP--R', hp_res, 1),
        ('BT--PE', bt_res, 0), ('BT--NC', bt_res, 1), ('BT--ET', bt_res, 2),
    ]
    model_definitions = [
        ("UNet", "bb_pad", "bb_up"),
        ("UNet+Aug", "bb-aug_pad", "bb-aug_up"),
        ("nnUNet", "nn_pad", "nn_up"),
    ]
    rare_unet_def = ("RARE-UNet", "our")
    scales_config = [
        (0, "1"), (1, "1/2"), (2, "1/4"), (3, "1/8"), ('overall', "Overall")
    ]

    # --- PASS 1: Collect all data, tagging each row with its scale group ---
    all_rows_data = []

    for scale, scale_name in scales_config:
        current_group_data = []

        # --- Define the rows for the current scale group ---
        # For scale (1), only show the base models
        if scale == 0:
            for model_name, pad_key, _ in model_definitions:
                current_group_data.append({'label': f"{model_name} ({scale_name})", 'model_key': pad_key})
        # For other scales (not 'Overall'), show both Pad and Up variants
        elif scale != 'overall':
            for model_name, pad_key, up_key in model_definitions:
                current_group_data.append({'label': f"{model_name}--Pad ({scale_name})", 'model_key': pad_key})
                current_group_data.append({'label': f"{model_name}--Up ({scale_name})", 'model_key': up_key})
        # For the 'Overall' group, also show both Pad and Up variants
        elif scale == 'overall':
            for model_name, pad_key, up_key in model_definitions:
                current_group_data.append({'label': f"{model_name}--Pad ({scale_name})", 'model_key': pad_key})
                current_group_data.append({'label': f"{model_name}--Up ({scale_name})", 'model_key': up_key})

        # Add RARE-UNet to every group
        current_group_data.append({'label': f"{rare_unet_def[0]} ({scale_name})", 'model_key': rare_unet_def[1]})
        
        # --- Calculate and store the data for each defined row ---
        for row_info in current_group_data:
            row_data = {'label': row_info['label'], 'scale_group': scale_name, 'means': [], 'stds': []}
            model_key = row_info['model_key']
            
            for _, res_dict, class_idx in columns:
                if scale != 'overall':
                    class_scores = np.array(res_dict[model_key][scale])[:, class_idx]
                    mean, std = np.mean(class_scores), np.std(class_scores)
                else: # Handle 'Overall' calculation for this specific row
                    means_per_scale = []
                    for s in range(4): # Average across all 4 scales [0, 1, 2, 3]
                        means_per_scale.append(np.mean(np.array(res_dict[model_key][s])[:, class_idx]))
                    mean, std = np.mean(means_per_scale), np.std(means_per_scale)
                
                row_data['means'].append(mean)
                row_data['stds'].append(std)
            all_rows_data.append(row_data)

    # --- PASS 2: Analyze and print, group by group ---
    print("\n\n" + "="*20 + " LaTeX Table for Per-Class DSC (Grouped) " + "="*20)
    
    for idx, (scale, scale_name) in enumerate(scales_config):
        group_rows = [row for row in all_rows_data if row['scale_group'] == scale_name]
        if not group_rows: continue

        group_means_array = np.array([data['means'] for data in group_rows])
        max_vals_for_group, second_max_vals_for_group = [], []
        
        for j in range(group_means_array.shape[1]):
            col_values_rounded = np.round(group_means_array[:, j], 3)
            unique_sorted_desc = np.unique(col_values_rounded)[::-1]
            max_vals_for_group.append(unique_sorted_desc[0] if len(unique_sorted_desc) > 0 else -1.0)
            second_max_vals_for_group.append(unique_sorted_desc[1] if len(unique_sorted_desc) > 1 else -1.0)

        for row_data in group_rows:
            latex_parts = []
            for j in range(len(columns)):
                mean, std = row_data['means'][j], row_data['stds'][j]
                mean_for_comparison = round(mean, 3)
                mean_str_for_display = f"{mean:.3f}"
                
                if mean_for_comparison == max_vals_for_group[j]:
                    formatted_mean = f"\\textbf{{{mean_str_for_display}}}"
                elif mean_for_comparison == second_max_vals_for_group[j]:
                    formatted_mean = f"\\underline{{{mean_str_for_display}}}"
                else:
                    formatted_mean = mean_str_for_display
                
                latex_parts.append(f"{formatted_mean}$\\pm${std:.3f}")
            
            print(f"{row_data['label']} & " + " & ".join(latex_parts) + " \\\\")

        if idx < len(scales_config) - 1:
            print("\\midrule")
            
    print("="*20 + " End of Table " + "="*20 + "\n")


# --- How to use ---
# At the end of your script, after `hp_res` and `bt_res` are populated, call:
generate_class_table_grouped(hp_res, bt_res)

KeyError: 'bb_pad'